In [1]:
import streamlit as st
import json
import matplotlib.pyplot as plt

# Function to plot lace based on nodes and paths
def plot_lace(data):
    plt.figure(figsize=(10, 6))
    nodes = data['nodes']
    paths = data['paths']
    for path in paths:
        x = [nodes[str(node)][0] for node in path['path']]
        y = [nodes[str(node)][1] for node in path['path']]
        plt.plot(x, y, marker='o')
    plt.grid(True)
    plt.show()

# Streamlit UI components
st.title('Lace Pattern Editor')

# Upload JSON file
uploaded_file = st.file_uploader("Choose a JSON file")
if uploaded_file is not None:
    data = json.load(uploaded_file)
    # Display JSON editor
    json_editor = st.text_area("Edit JSON:", value=json.dumps(data, indent=4), height=300)
    data = json.loads(json_editor)
    # Plotting
    st.write("Lace Pattern Visualization:")
    fig, ax = plt.subplots()
    plot_lace(data)
    st.pyplot(fig)


2024-07-08 11:05:36.478 
  command:

    streamlit run /appl/scibuilder-mamba/aalto-rhel9/prod/software/scicomp-python-env/2024-01/f56a564/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
import subprocess

# Replace 'app.py' with the path to your Streamlit script if it's not in the same directory
subprocess.run(["streamlit", "run", "/appl/scibuilder-mamba/aalto-rhel9/prod/software/scicomp-python-env/2024-01/f56a564/lib/python3.11/site-packages/ipykernel_launcher.py"])